In [15]:
!git clone https://github.com/emanhamed/Houses-dataset

fatal: destination path 'Houses-dataset' already exists and is not an empty directory.


In [0]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os

In [0]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam

In [18]:
cwd = os.getcwd()
for i, file in enumerate(os.listdir(cwd)):
    print(i, file)

0 .config
1 housing_prices_stats_model.h5
2 Houses-dataset
3 housing_prices_model.h5
4 sample_data


In [19]:
house_jpg_dir = os.path.join(cwd, 'Houses-dataset', 'Houses Dataset')
print(f"{len(os.listdir(house_jpg_dir))} JPG files")

houses_info = os.path.join(cwd, 'Houses-dataset', 'Houses Dataset', 'HousesInfo.txt')

2141 JPG files


In [0]:
# Loading dataset and removing datapoints where there are less than 25 houses
# per zip code.
def load_house_attributes(inputPath):
    cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
    df = pd.read_csv(inputPath, sep=' ', header=None, names=cols)
    
    zipcodes = df['zipcode'].value_counts().keys().tolist()
    counts   = df['zipcode'].value_counts().tolist()
    #df.head()
    
    for (zipcode, count) in zip(zipcodes, counts):
      
        if count < 25:
            idxs = df[df['zipcode'] == zipcode].index
            df.drop(idxs, inplace=True)
            
    return df

In [0]:
def process_house_attributes(df, train, test):
    continuous = ['bedrooms', 'bathrooms', 'area']
    
    cs               = MinMaxScaler()
    train_continuous = cs.fit_transform(train[continuous])
    test_continuous  = cs.transform(test[continuous])
    
    zip_binarizer     = LabelBinarizer().fit(df['zipcode'])
    train_categorical = zip_binarizer.transform(train['zipcode'])
    test_categorical  = zip_binarizer.transform(test['zipcode'])
    
    train_X = np.hstack([train_categorical, train_continuous])
    test_X  = np.hstack([test_categorical, test_continuous])
    
    return (train_X, test_X)

In [0]:
def create_mlp(dim, regress=False):
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation='relu'))
    model.add(Dense(4, activation='relu'))
    
    if regress:
        model.add(Dense(1, activation='linear'))
        
    return model

In [23]:
df = load_house_attributes(houses_info)

(train, test) = train_test_split(df, test_size=0.25, random_state=42)

max_price = train['price'].max()
train_y   = train['price'] / max_price
test_y    = test['price'] / max_price
print(f"Max price: {max_price}")

Max price: 5858000.0


In [0]:
(train_X, test_X) = process_house_attributes(df, train, test)

In [25]:
model = create_mlp(train_X.shape[1], regress=True)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 5         
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________


In [26]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

model.fit(train_X,
          train_y,
          validation_data=(test_X, test_y),
          epochs=200,
          batch_size=8)


Train on 271 samples, validate on 91 samples
Epoch 1/200
271/271 [==============================] - 0s 1ms/step - loss: 387.7268 - val_loss: 180.6732
Epoch 2/200
271/271 [==============================] - 0s 183us/step - loss: 123.8931 - val_loss: 98.3965
Epoch 3/200
271/271 [==============================] - 0s 196us/step - loss: 76.7719 - val_loss: 78.6968
Epoch 4/200
271/271 [==============================] - 0s 202us/step - loss: 64.3747 - val_loss: 67.5734
Epoch 5/200
271/271 [==============================] - 0s 169us/step - loss: 58.6875 - val_loss: 59.8533
Epoch 6/200
271/271 [==============================] - 0s 179us/step - loss: 55.5463 - val_loss: 53.6864
Epoch 7/200
271/271 [==============================] - 0s 179us/step - loss: 47.6688 - val_loss: 48.0923
Epoch 8/200
271/271 [==============================] - 0s 170us/step - loss: 44.5377 - val_loss: 46.1539
Epoch 9/200
271/271 [==============================] - 0s 181us/step - loss: 43.0114 - val_loss: 47.7090
Epoch 10/

In [0]:
model.save("housing_prices_stats_model.h5")

In [28]:
import locale

preds = model.predict(test_X)

diff = preds.flatten() - test_y
percent_diff = (diff / test_y) * 100
abs_percent_diff = np.abs(percent_diff)

mean = np.mean(abs_percent_diff)
std  = np.std(abs_percent_diff)

print("avg. house price: ${:.2f}, std house price: ${:.2f}".format(
    df["price"].mean(), df["price"].std()))

print("mean: {:.2f}%, std: {:.2f}%".format(mean, std))

avg. house price: $533388.27, std house price: $493403.08
mean: 23.97%, std: 23.06%


In [0]:
from google.colab import files
files.download("housing_prices_stats_model.h5")